In [1]:
from pathlib import Path
import torch
import numpy as np
import os

from torch import cuda
import pytorch_lightning as pl
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
import seaborn as sns
import matplotlib.pyplot as plt

os.chdir("..")
from utils import io
from models import SegmentationModel

/home/students/s265780/.conda-envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
#reports/unet_2/unet_2/epoch=85-val_rmse=7.71.ckpt  first ok
#reports/unet_2/unet_2/epoch=86-val_rmse=8.59.ckpt add torch.where()
#reports/unet_2/unet_2/epoch=72-val_rmse=7.35.ckpt add mask mult
#reports/unet/unet/epoch=97-val_rmse=7.87.ckpt unetL1
#reports/unet/unet/epoch=85-val_rmse=7.90.ckpt unet
#reports/unet/unet/epoch=0-val_rmse=24.06.ckpt unet 24.19 rmse
#reports/unet/unet/epoch=85-val_rmse=7.06.ckpt unet 5.99 rmse

model = SegmentationModel.load_from_checkpoint(Path("reports/unet/unet/epoch=90-val_rmse=7.14.ckpt"))
model.eval();

In [54]:
input_path = Path("/home/students/s265780/data/24h_10mmMAX_OI")
test_date_9 = [
    "20190807", "20181030", "20200329", "20200501", "20190505", "20200128", "20180825",
    "20210807", "20210724", "20201006", "20210628", "20220518", "20190702", "20190802",
    "20210617", "20190806", "20200519", "20190618", "20180817", "20180801", "20190703",
    "20190714", "20220331", "20190906", "20181123", "20180720", "20200617", "20220524",
    "20190919", "20210730", "20190424", "20191218", "20200609", "20180607", "20181124",
    "20200801", "20210727", "20200420", "20211031", "20190828", "20190819", "20190412",
    "20200627", "20210101", "20181029", "20200701", "20200616", "20200426", "20180714",
    "20220528", "20200921", "20190908", "20200830", "20180813", "20191022", "20210831",
    "20180830", "20210712", "20191014", "20180820", "20190710", "20210129", "20211004"
]

indices_one, indices_zero, mask = io.get_mask_indices(input_path, True) #why False?
case_study_max=483.717752
available_models = ["bol00", "e1000", "c2200", "c5m00"]
x, y, in_features, out_features = io.load_data('unet', input_path, test_date_9, case_study_max, indices_one, indices_zero, available_models)
x[:, 1]=np.zeros((96,128))

In [55]:
logit_seg, logit_class = model(torch.from_numpy(x).to('cuda'), None)

#logit_seg_numpy = logit_seg.squeeze().to('cpu').detach().numpy()
logit_class = logit_class.squeeze().to('cpu').detach().numpy()

##seg = torch.round(torch.sigmoid(logit_seg)).squeeze().to('cpu').detach().numpy()
#seg = logit_seg.sigmoid().gt(.5).squeeze().to('cpu').detach().numpy()

y_seg = torch.heaviside(torch.from_numpy(y).to('cuda'), torch.tensor([0]).float().to('cuda')).squeeze().to('cpu').detach().numpy()
y_seg_num = np.squeeze(np.where(y>0.001, 1, 0))

RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

In [ ]:
#plt.imshow(seg, cmap='gray') #modello

In [ ]:
#plt.imshow(y_seg_num, cmap='gray') #ground thruth

In [ ]:
#plt.imshow(y_seg_num-seg, cmap='gray') #parti bianche mancano nel modello

In [ ]:
m=4
norm='linear'
vmin=0
vmax=1/20
fig, axs = plt.subplots(2, 3, figsize=(15, 7))
axs[0, 0].imshow(x[m][0], cmap='gray', norm=norm, vmin=vmin, vmax=vmax)
axs[0, 0].set_title('bol00')
axs[0, 1].imshow(x[m][1], cmap='gray', norm=norm, vmin=vmin, vmax=vmax)
axs[0, 1].set_title('c2200')
axs[1, 0].imshow(x[m][2], cmap='gray', norm=norm, vmin=vmin, vmax=vmax)
axs[1, 0].set_title('c5m00')
axs[1, 1].imshow(x[m][3], cmap='gray', norm=norm, vmin=vmin, vmax=vmax)
axs[1, 1].set_title('e1000')
axs[0, 2].imshow(y[m], cmap='gray', norm=norm, vmin=vmin, vmax=vmax)
axs[0, 2].set_title('ground_thruth')
axs[1, 2].imshow(logit_class[m], cmap='gray', norm=norm, vmin=vmin, vmax=vmax)
axs[1, 2].set_title('model_pred')

In [ ]:
histo = (logit_class*case_study_max*mask).flatten()

plt.hist(histo[histo>0], bins=np.linspace(0,10, 100))
plt.show()

In [ ]:
histo = (y*case_study_max*mask).flatten()

plt.hist(histo[histo>0], bins=np.linspace(0,10, 100))
plt.show()

In [ ]:
y_r = np.reshape(y*mask, (63, 128*96))
logit_class_r = np.reshape(logit_class*mask, (63, 128*96))
RMSE = np.sqrt(np.square(y_r*case_study_max-logit_class_r*case_study_max).sum(0)/63)
plt.hist(RMSE[RMSE>0], bins=np.linspace(0,20, 100))
plt.title("modello")
plt.show()
RMSE.sum(), np.shape(RMSE)

In [ ]:
x1_r = np.reshape(x, (63, 4, 128*96)).mean(1)
RMSE = np.sqrt(np.square(y_r*case_study_max-x1_r*case_study_max).sum(0)/63)
plt.hist(RMSE[RMSE>0], bins=np.linspace(0,20, 100))
plt.title("modello matematico")
plt.show()
RMSE.sum(), np.shape(RMSE)

In [ ]:
np.sqrt(np.square(y*case_study_max-logit_class*case_study_max).mean())

In [ ]:
from sklearn.metrics import root_mean_squared_error
sum=0
for i in range(63):
    sum+=np.square(root_mean_squared_error(y_r[i]*case_study_max, logit_class_r[i]*case_study_max))
sum/=63
np.sqrt(sum)

In [ ]:
import torch.nn as nn

In [56]:
(nn.MSELoss()(torch.tensor(logit_class).cpu(), torch.tensor(y).cpu())*case_study_max**2)**.5

tensor(5.0207)

In [36]:

trainer = pl.Trainer(
        accelerator='gpu' if cuda.is_available() else 'cpu',
        callbacks=[],
		log_every_n_steps=1
    )

model = SegmentationModel.load_from_checkpoint(Path("reports/unet/unet/epoch=85-val_rmse=7.04.ckpt"))
trainer.test(model)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]sum pred: 5846.00634765625, sum pred*mask: 5846.00634765625, sum y: 5164.935546875, sum y*mask: 5164.935546875
y shape: torch.Size([63, 96, 128]), y_hat shape: torch.Size([63, 96, 128]), x0 shape: torch.Size([63, 96, 128]), x1 shape: torch.Size([63, 96, 128]), x2 shape: torch.Size([63, 96, 128]), x3 shape: torch.Size([63, 96, 128])
np.rmse: 13.77782154083252
torch loss: 5.517566204071045
tensor(5917.0889, device='cuda:0')
tensor(5943.9141, device='cuda:0')
tensor(5685.8018, device='cuda:0')
tensor(5465.8862, device='cuda:0')
Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  4.08it/s]


/home/students/s265780/rainfall-prediction/models.py:168: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  print(torch.tensor(x[:, i]).sum())


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        rmse NWP 0         │     8.034293174743652     │
│        rmse NWP 1         │     5.912471294403076     │
│        rmse NWP 2         │     8.256181716918945     │
│        rmse NWP 3         │     7.918542385101318     │
│         test rmse         │     5.517565727233887     │
└───────────────────────────┴───────────────────────────┘

[{'test rmse': 5.517565727233887,
  'rmse NWP 0': 8.034293174743652,
  'rmse NWP 1': 5.912471294403076,
  'rmse NWP 2': 8.256181716918945,
  'rmse NWP 3': 7.918542385101318}]

In [51]:
for i in range(4):
    print(np.sqrt(np.square(y*case_study_max-x[:,i]*case_study_max).mean()))

8.034293
5.912471
8.256181
7.9185424


In [52]:
for i in range(4):
    print(torch.tensor(x[:, i]).sum())

tensor(5917.0884)
tensor(5943.9141)
tensor(5685.8018)
tensor(5465.8862)


In [53]:
torch.bernoulli(torch.tensor(.1))

tensor(0.)